# Projet statistique descriptive 



In [1]:
%reload_ext autoreload
%autoreload 2

# module
import sys
sys.path.append("../")
from src import * 

# chemin vers la donnee 
data_dir = 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive/'
filelist = glob.glob(data_dir + '*.h5')
filelist

['C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_01.h5',
 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_01_dask.h5',
 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_02.h5',
 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_03.h5']

In [3]:
from tabata import Opset

In [8]:
df_opset1=Opset('C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_01.h5')
df_opset2=Opset('C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_02.h5')
df_opset3=Opset('C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\\Aircraft_03.h5')

In [5]:
df_opset1

OPSET 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\Aircraft_01.h5' de 1002 signaux.
        position courante : sigpos  = 0
        variable courante : colname = ALT [ft]
        phase surlignée   : phase   = None

In [9]:
df_opset1.plot()

In [10]:
df_opset2.plot()

In [75]:
df_opset2.plot()

In [45]:
def eliminate_records(df):
    alt_min=df['ALT [ft]'].min()
    alt_max=df['ALT [ft]'].max()
    alt_range=alt_max-alt_min
    bol=False
    
    if alt_range<15000:
        bol=True
    return bol
    
    

In [74]:
#on cree notre dataframe avec DASK
ddf1 = lire_hdf_dask(filelist[0], repertoire= data_dir)
#on récupère une liste de booleen où chaque composante fait reférance à une partition 
I = ddf1.map_partitions(lambda df: eliminate_records(df)).compute()
#on sélectione dans un nouveau dataframe les partitions qui ne sont pas mauvaise
DDF1=ddf1.partitions[I==False]
# on normalise toute les données excepté les variables booleen 
ddf_norm1 = DDF1.map_partitions(lambda df : normalize(df))

ddf2 = lire_hdf_dask(filelist[2], repertoire= data_dir)
I = ddf2.map_partitions(lambda df: eliminate_records(df)).compute()
DDF2=ddf2.partitions[I==False]
ddf_norm2 = DDF2.map_partitions(lambda df : normalize(df))

ddf3 = lire_hdf_dask(filelist[3], repertoire= data_dir)
I = ddf3.map_partitions(lambda df: eliminate_records(df)).compute()
DDF3=ddf3.partitions[I==False]
ddf_norm3 = DDF3.map_partitions(lambda df : normalize(df))

C:\Users\felix\PycharmProjects\pythonProject1\projet_lacaille\stats_descriptive_project\notebooks\..\src\manipulate_dataframe.py:19: RuntimeWarning:

invalid value encountered in divide



Ce HDFStore n'est pas partitionnable et ne peut être utilisé de manière monolithique qu'avec pandas.
Création d'un nouveau fichier de données: 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\Aircraft_02_dask.h5'
Lecture du fichier de données: 'C:/Users/felix/PycharmProjects/pythonProject1/MACS3-Statistiques-Descriptives-TDs/data/archive\Aircraft_02_dask.h5'


C:\Users\felix\PycharmProjects\pythonProject1\projet_lacaille\stats_descriptive_project\notebooks\..\src\manipulate_dataframe.py:19: RuntimeWarning:

invalid value encountered in divide

C:\Users\felix\PycharmProjects\pythonProject1\projet_lacaille\stats_descriptive_project\notebooks\..\src\manipulate_dataframe.py:19: RuntimeWarning:

invalid value encountered in divide



In [53]:
df=ddf.partitions[1].compute()
df['ALT [ft]'].min()

-134.462575892